In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
# from art.estimators.classification import PyTorchClassifier

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./TRADES.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./TRADES.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def3/y_pred_{attack_name}{eps}_Def3.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')


In [11]:
def squared_l2_norm(x):
    flattened = x.view(x.unsqueeze(0).shape[0], -1)
    return (flattened ** 2).sum(1)


def l2_norm(x):
    return squared_l2_norm(x).sqrt()

def trades_loss(model,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=1.0,
                distance='l_inf'):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(size_average=False)
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()
    if distance == 'l_inf':
        for _ in range(perturb_steps):
            x_adv.requires_grad_()
            with torch.enable_grad():
                loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(model(x_natural), dim=1))
            grad = torch.autograd.grad(loss_kl, [x_adv])[0]
            x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
            x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
            x_adv = torch.clamp(x_adv, 0.0, 1.0)
    elif distance == 'l_2':
        delta = 0.001 * torch.randn(x_natural.shape).cuda().detach()
        delta = Variable(delta.data, requires_grad=True)

        # Setup optimizers
        optimizer_delta = optim.SGD([delta], lr=epsilon / perturb_steps * 2)

        for _ in range(perturb_steps):
            adv = x_natural + delta

            # optimize
            optimizer_delta.zero_grad()
            with torch.enable_grad():
                loss = (-1) * criterion_kl(F.log_softmax(model(adv), dim=1),
                                           F.softmax(model(x_natural), dim=1))
            loss.backward()
            # renorming gradient
            grad_norms = delta.grad.view(batch_size, -1).norm(p=2, dim=1)
            delta.grad.div_(grad_norms.view(-1, 1, 1, 1))
            # avoid nan or inf if gradient is 0
            if (grad_norms == 0).any():
                delta.grad[grad_norms == 0] = torch.randn_like(delta.grad[grad_norms == 0])
            optimizer_delta.step()

            # projection
            delta.data.add_(x_natural)
            delta.data.clamp_(0, 1).sub_(x_natural)
            delta.data.renorm_(p=2, dim=0, maxnorm=epsilon)
        x_adv = Variable(x_natural + delta, requires_grad=False)
    else:
        x_adv = torch.clamp(x_adv, 0.0, 1.0)
    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    # calculate robust loss
    logits = model(x_natural)
    loss_natural = F.cross_entropy(logits, y)
    loss_robust = (1.0 / batch_size) * criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                                    F.softmax(model(x_natural), dim=1))
    loss = loss_natural + beta * loss_robust
    return loss

In [12]:
# # Adversarial training loop
# for epoch in range(10):
#     model.train()
#     train_loss = 0.0
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)

#         optimizer.zero_grad()
#         # calculate robust loss - TRADES loss
#         loss = trades_loss(model=model,
#                            x_natural=data,
#                            y=target,
#                            optimizer=optimizer,
#                            step_size=0.01,
#                            epsilon=0.3,
#                            perturb_steps=10,
#                            beta=1.0,
# 			   distance='l_inf')
#         loss.backward()
#         optimizer.step()
#         # print progress
#         if batch_idx % 1000 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
            
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break


In [13]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/TRADES/TRADES.pt"))

/tmp/ipykernel_3515/2152490654.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/TR

<All keys matched successfully>

In [14]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [15]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./TRADES.csv", mode='a', index=False, header=False)

In [16]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [17]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(2277, 56) (2277,)
Accuracy: 0.6315327184892402

macro
Precision: 0.6400584637037838
Recall: 0.5818790438320139
F1 Score: 0.5585183024889

weighted
Precision: 0.6380084017265427
Recall: 0.6315327184892402
F1 Score: 0.5868251873097663

start RF 10 BIM
(182, 56) (182,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9505494505494505

macro
Precision: 0.5
Recall: 0.47527472527472525
F1 Score: 0.48732394366197185

weighted
Precision: 1.0
Recall: 0.9505494505494505
F1 Score: 0.9746478873239438

start RF 10 FGSM
(167, 56) (167,)
Accuracy: 0.9520958083832335

macro
Precision: 0.6
Recall: 0.9757575757575758
F1 Score: 0.6542443064182194

weighted
Precision: 0.9904191616766468
Recall: 0.9520958083832335
F1 Score: 0.9674687891298148

start RF 10 PGD
(182, 56) (182,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9505494505494505

macro
Precision: 0.5
Recall: 0.47527472527472525
F1 Score: 0.48732394366197185

weighted
Precision: 1.0
Recall: 0.9505494505494505
F1 Score: 0.9746478873239438

start RF 10 DF
(2075, 56) (2075,)
Accuracy: 0.9224096385542169

macro
Precision: 0.6935342465753425
Recall: 0.919326039054783
F1 Score: 0.7532864978314026

weighted
Precision: 0.9639705033833965
Recall: 0.9224096385542169
F1 Score: 0.9364868266353359

start RF 10 AutoPGD
(3743, 56) (3743,)
Accuracy: 0.8559978626769971

macro
Precision: 0.6885565768659828
Recall: 0.7474958413853081
F1 Score: 0.7108366197526281

weighted
Precision: 0.87994704052633
Recall: 0.8559978626769971
F1 Score: 0.865686226175343

start RF 10 ZOO
(5895, 56) (5895,)
Accuracy: 0.7375742154368109

macro
Precision: 0.7272983877662842
Recall: 0.6719202421689918
F1 Score: 0.6809195371894996

weighted
Precision: 0.7326581839448162
Recall: 0.7375742154368109
F1 Score: 0.719761291237475

start RF 10 CaFA
(476, 56) (476,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.22899159663865545

macro
Precision: 0.5
Recall: 0.11449579831932773
F1 Score: 0.18632478632478633

weighted
Precision: 1.0
Recall: 0.22899159663865545
F1 Score: 0.37264957264957266

start RF 10 SINIFGSM
(12, 56) (12,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 VNIFGSM
(153, 56) (153,)
Accuracy: 0.9607843137254902

macro
Precision: 0.5
Recall: 0.4803921568627451
F1 Score: 0.49

weighted
Precision: 1.0
Recall: 0.9607843137254902
F1 Score: 0.98



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(2168, 56) (2168,)
Accuracy: 0.6549815498154982

macro
Precision: 0.672721659990553
Recall: 0.6206038046183479
F1 Score: 0.6083618307927123

weighted
Precision: 0.668600810674766
Recall: 0.6549815498154982
F1 Score: 0.6259377141676128

start RF 10 BIM
(330, 56) (330,)
Accuracy: 0.9090909090909091

macro
Precision: 0.4918032786885246
Recall: 0.46153846153846156
F1 Score: 0.47619047619047616

weighted
Precision: 0.9687034277198212
Recall: 0.9090909090909091
F1 Score: 0.937950937950938

start RF 10 FGSM
(331, 56) (331,)
Accuracy: 0.8882175226586103

macro
Precision: 0.5044623655913979
Recall: 0.5160603715170279
F1 Score: 0.4959460015639791

weighted
Precision: 0.9538410811162006
Recall: 0.8882175226586103
F1 Score: 0.9191156219229134

start RF 10 PGD
(330, 56) (330,)
Accuracy: 0.9090909090909091

macro
Precision: 0.4918032786885246
Recall: 0.46153846153846156
F1 Score: 0.47619047619047616

weighted
Precision: 0.9687034277198212
Recall: 0.9090909090909091
F1 Score: 0.9

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8955696202531646

macro
Precision: 0.49475524475524474
Recall: 0.4520766773162939
F1 Score: 0.4724540901502504

weighted
Precision: 0.980116402584757
Recall: 0.8955696202531646
F1 Score: 0.9359375330191669



In [19]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
(1222, 56) (1222,)
Accuracy: 0.7013093289689034

macro
Precision: 0.7153912056373022
Recall: 0.6413877266387726
F1 Score: 0.6392029023607971

weighted
Precision: 0.7103834794423902
Recall: 0.7013093289689034
F1 Score: 0.6717873392005531

start RF 10 BIM
(474, 56) (474,)
Accuracy: 0.8565400843881856

macro
Precision: 0.5280868902439024
Recall: 0.6144409937888199
F1 Score: 0.5250221043324492

weighted
Precision: 0.9514687532160131
Recall: 0.8565400843881856
F1 Score: 0.8983984152033038

start RF 10 FGSM
(480, 56) (480,)
Accuracy: 0.84375

macro
Precision: 0.5242088226178587
Recall: 0.6077559779276518
F1 Score: 0.5159663865546218

weighted
Precision: 0.9515242432590654
Recall: 0.84375
F1 Score: 0.8910504201680671

start RF 10 PGD
(474, 56) (474,)
Accuracy: 0.8565400843881856

macro
Precision: 0.5280868902439024
Recall: 0.6144409937888199
F1 Score: 0.5250221043324492

weighted
Precision: 0.9514687532160131
Recall: 0.8565400843881856
F1 Score: 0.8983984152033038

start 

In [16]:
epsilon = 0

Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9080914687774846

macro
Precision: 0.9218471904398677
Recall: 0.8093963434059317
F1 Score: 0.8492097460685601

weighted
Precision: 0.9110156410554
Recall: 0.9080914687774846
F1 Score: 0.9010886460204063

start BIM
Accuracy: 0.6875241405948243

macro
Precision: 0.6654957703487019
Recall: 0.6358873047509281
F1 Score: 0.6393768600405819

weighted
Precision: 0.6769973087957925
Recall: 0.6875241405948243
F1 Score: 0.6724081339091901

start FGSM
Accuracy: 0.7704194260485652

macro
Precision: 0.7282666260641608
Recall: 0.7224662112341655
F1 Score: 0.7251767754074981

weighted
Precision: 0.7677543385890665
Recall: 0.7704194260485652
F1 Score: 0.7689425168872681

start PGD
Accuracy: 0.6875241405948243

macro
Precision: 0.6654957703487019
Recall: 0.6358873047509281
F1 Score: 0.6393768600405819

weighted
Precision: 0.6769973087957925
Recall: 0.6875241405948243
F1 Score: 0.6724081339091901

start DF
Accuracy: 0.854607250755287

macro
Precision: 0.6195335510308618
Recall:

In [20]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(31571, 56) (31571,)
Accuracy: 0.9607234487345983

macro
Precision: 0.7992934132499684
Recall: 0.9576028347157157
F1 Score: 0.8583075185541388

weighted
Precision: 0.97415454796542
Recall: 0.9607234487345983
F1 Score: 0.9648138282716293

start RF 10 BIM
(47197, 56) (47197,)
Accuracy: 0.9543191304532068

macro
Precision: 0.7711405574467347
Recall: 0.9620308955444337
F1 Score: 0.8362858367590207

weighted
Precision: 0.9736238780142662
Recall: 0.9543191304532068
F1 Score: 0.9602274220851603

start RF 10 FGSM
(47514, 56) (47514,)
Accuracy: 0.9543191304532068

macro
Precision: 0.7711405574467347
Recall: 0.9620308955444337
F1 Score: 0.8362858367590207

weighted
Precision: 0.9736238780142662
Recall: 0.9543191304532068
F1 Score: 0.9602274220851603

start RF 10 DF
(1926, 56) (1926,)
Accuracy: 0.9029075804776739

macro
Precision: 0.8303242261338195
Recall: 0.8581552874141718
F1 Score: 0.843118276993843

weighted
Precision: 0.9078306253809963
Recall: 0.9029075804776739
F1 Sco

In [21]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start RF 10 baseline
(285, 56) (285,)
Accuracy: 0.9157894736842105

macro
Precision: 0.9562043795620438
Recall: 0.6571428571428571
F1 Score: 0.7162296714238301

weighted
Precision: 0.9231655781790243
Recall: 0.9157894736842105
F1 Score: 0.8957499956329591

start RF 10 BIM
(282, 56) (282,)
Accuracy: 0.9148936170212766

macro
Precision: 0.7272727272727273
Recall: 0.9541984732824427
F1 Score: 0.7885

weighted
Precision: 0.9613152804642165
Recall: 0.9148936170212766
F1 Score: 0.9288085106382978

start RF 10 FGSM
(288, 56) (288,)
Accuracy: 0.9097222222222222

macro
Precision: 0.717391304347826
Recall: 0.9514925373134329
F1 Score: 0.7775401069518717

weighted
Precision: 0.9607487922705314
Recall: 0.9097222222222222
F1 Score: 0.925203010497128

start RF 10 PGD
(282, 56) (282,)
Accuracy: 0.9148936170212766

macro
Precision: 0.7272727272727273
Recall: 0.9541984732824427
F1 Score: 0.7885

weighted
Precision: 0.9613152804642165
Recall: 0.9148936170212766
F1 Score: 0.9288085106382978

start RF 10 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9833531510107015

macro
Precision: 0.8
Recall: 0.9914634146341463
F1 Score: 0.8706949569495694

weighted
Precision: 0.9900118906064209
Recall: 0.9833531510107015
F1 Score: 0.9853623417328109

start RF 10 ZOO
(1079, 56) (1079,)
Accuracy: 0.995366079703429

macro
Precision: 0.965699548678272
Recall: 0.9758234519104085
F1 Score: 0.9707017991647704

weighted
Precision: 0.995426072688313
Recall: 0.995366079703429
F1 Score: 0.9953909931181144

start RF 10 CaFA
(5, 56) (5,)
Accuracy: 0.4

macro
Precision: 0.3333333333333333
Recall: 0.25
F1 Score: 0.2857142857142857

weighted
Precision: 0.5333333333333333
Recall: 0.4
F1 Score: 0.45714285714285713

start RF 10 SINIFGSM
(11, 56) (11,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 VNIFGSM
(300, 56) (300,)
Accuracy: 0.8366666666666667

macro
Precision: 0.5775862068965517
Recall: 0.9158075601374571
F1 Score: 0.588362129315897

weighted
Precision: 0.974655172

In [22]:
epsilon = 0

Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]


active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
Accuracy: 0.7309368191721133

macro
Precision: 0.7266681250326943
Recall: 0.7372759856630824
F1 Score: 0.7263382506822053

weighted
Precision: 0.747798161022818
Recall: 0.7309368191721133
F1 Score: 0.7339896503544893

start RF 10 BIM
Accuracy: 0.6414950419527079

macro
Precision: 0.5246923169868641
Recall: 0.5289165150596321
F1 Score: 0.5241636548975999

weighted
Precision: 0.6732947367078067
Recall: 0.6414950419527079
F1 Score: 0.6555531804938729

start RF 10 FGSM
Accuracy: 0.7229241877256317

macro
Precision: 0.5648891266868795
Recall: 0.6333592188778099
F1 Score: 0.5601741945827967

weighted
Precision: 0.8460937816898552
Recall: 0.7229241877256317
F1 Score: 0.7685617970787413

start RF 10 PGD
Accuracy: 0.6414950419527079

macro
Precision: 0.5246923169868641
Recall: 0.5289165150596321
F1 Score: 0.5241636548975999

weighted
Precision: 0.6732947367078067
Recall: 0.6414950419527079
F1 Score: 0.6555531804938729

start RF 10 DF
Accuracy: 0.9138038725796377

macro
Prec

In [15]:
# torch.save(model.state_dict(), "./TRADES.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")